<a href="https://colab.research.google.com/github/labilello/EA2-SOA/blob/master/HPC/Bilello_Leandro__ejercicio_1GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introducción

El ejemplo desarrollado en este notebook, rota una imágen parametrizada 180 grados. El calculo de la rotación para una imagen parametrizada siguiendo la logica:

**destino[anchoImg - 1 - x][altoImg - 1 - y] <- origen[x][y]**


Donde "x" refiere a una columna de la matriz e "y" refiere una fila de la misma.

# 2. Armado del Ambiente

Solicitamos al sistema la instalación de "pycuda", el modulo para python encargado de facilitar al programador el acceso a CUDA de Nvidia.

In [ ]:
!pip install pycuda

# 3 Desarrallo
A continuacion se encuentra el algoritmo de rotación de imágen para GPU. Ejecute el modulo a continuacion para ver los resultados.

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
cant_elements = 256 #@param {type: "number"}
# --------------------------------------------

from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy


tiempo_total = datetime.now()
# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# CPU - Defino la memoria de los vectores en cpu.
x_cpu = numpy.random.randn( cant_elements )
x_cpu = x_cpu.astype( numpy.int32 )

#tiempo_ini_cpu = datetime.now()

r_cpu = numpy.array( [0] )
r_cpu = r_cpu.astype( numpy.int32 )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )
r_gpu = cuda.mem_alloc( r_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )
cuda.memcpy_htod( r_gpu, r_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void reduce0(int n, int *input, int *output) {

  unsigned int tid = threadIdx.x;
  unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

  __syncthreads();

  for (unsigned int j = 1; j < n; j *= 2) {
    if (tid % (2 * j) == 0) {
    input[tid] += input[tid + j];
  }

  __syncthreads();
  }

  if (tid == 0) *output = input[0];
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("reduce0")


dim_hilo = 1024
dim_bloque = numpy.int( (cant_elements+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

tiempo_gpu = datetime.now()

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cant_elements), x_gpu, r_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, r_gpu )

# CPU - Informo el resutlado.
#print( "------------------------------------")
#print( "X: " )
#print( x_cpu )
#print( "------------------------------------")
#print( "R: " )
#print( r_cpu )


tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", cant_elements )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

# 4. Tabla de Pasos

# 5. Conclusión

# 6. Bibliografía